In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
from fastai.conv_learner import*
from fastai.dataset import*

from pathlib import Path
import json
from PIL import ImageDraw, ImageFont
from matplotlib import patches, patheffects

In [7]:
[i for i in PATH.iterdir()]

[PosixPath('data/pascal_2007/train'),
 PosixPath('data/pascal_2007/test'),
 PosixPath('data/pascal_2007/test.csv'),
 PosixPath('data/pascal_2007/train.csv'),
 PosixPath('data/pascal_2007/train.json'),
 PosixPath('data/pascal_2007/test.json'),
 PosixPath('data/pascal_2007/segmentation'),
 PosixPath('data/pascal_2007/valid.json')]

In [4]:
PATH = Path('data/pascal_2007')
list(PATH.iterdir())

[PosixPath('data/pascal_2007/train'),
 PosixPath('data/pascal_2007/test'),
 PosixPath('data/pascal_2007/test.csv'),
 PosixPath('data/pascal_2007/train.csv'),
 PosixPath('data/pascal_2007/train.json'),
 PosixPath('data/pascal_2007/test.json'),
 PosixPath('data/pascal_2007/segmentation'),
 PosixPath('data/pascal_2007/valid.json')]